In [136]:
###モンテカルロES法###
#初期化

import numpy as np
import copy

#0～399のsanumberに対して、状態行動の割り当て
def getStateAction(sanumber):
    
    if sanumber % 2==1:
        hit=True
    else:
        hit=False
        
    if sanumber>=200:
        usableA=True
    else:
        usableA=False
        
    n=sanumber // 2
    n=n % 100
    
    sumCards=(n % 10)+12
    dealerCard=(n // 10)+2
    
    sa=[sumCards, dealerCard, usableA, hit]
    return sa

#状態ナンバー(0～199)
def getStateNumber(sumCards, dealerCard, usableA):
    n=(sumCards-12)+10*(dealerCard-2)
    if usableA:
        n+=100
    return n

def pickCard():
    #1～13の整数
    c=np.random.randint(1,14)
    if c>10:
        c=10
    elif c==1:
        c=11
    return c


def playerAct(sanumber):
    #経由した状態行動対の番号の集合(初回訪問MCに対応)
    #リストを使えば逐一訪問MCになる。
    sanumbers=set([sanumber])
    
    sa=getStateAction(sanumber)
    
    sumCards=sa[0]
    dealerCard=sa[1]
    usableA=sa[2]
    hit=sa[3]

    while(hit):
        c=pickCard()
        sumCards+=c
        if c==11:
            usableA=True
        if sumCards>21 and usableA:
            sumCards-=10
            usableA=False
        if sumCards>21:
            break
            
        snumber=getStateNumber(sumCards, dealerCard, usableA)
        #乱数選択
        r=np.random.rand()
        if r<piH[snumber]:
            hit=True
            sanumber=snumber*2+1
        else:
            hit=False
            sanumber=snumber*2
            
        sanumbers.add(sanumber)
        
    result=[sumCards,dealerCard,sanumbers]
    return result
        
def dealerAct(dealerCard):
    usableA=False
    if dealerCard==11:
        usableA=True
    s=dealerCard

    while(s<17):
        c=pickCard()
        s+=c
        if c==11:
            usableA=True
        if s>21 and usableA:
            s-=10
            usableA=False
            
    return s

def getReturn(psum, dsum):
    if psum>21:
        r=-1
    elif dsum>21:
        r=1
    elif psum>dsum:
        r=1
    elif dsum>psum:
        r=-1
    else:
        r=0
        
    return r

In [137]:
Q=np.random.rand(200,2)*2-1 #Qの初期化　[-1,1]の乱数
piH=np.random.rand(200) #方策の初期化：piHの確率でHit
n=1000 #試行セット数

returns=[]
for i in range(400):
    ret=[] #appendはリストの方が速い？numPy配列は平均値計算が速いがAppendが遅い
    returns.append(ret)
    
table=np.zeros((n,3))

#400個の各状態行動対に対してシミュレーションを1セットとしてnセット
for i in range(n):
    
    piBfr=copy.deepcopy(piH)
    QBfr=copy.deepcopy(Q)
    
    for j in range(400):
        result=playerAct(j)
        psum=result[0]
        sanumbers=result[2]
        dsum=0
        if psum<=21:
            dealerCard=result[1]
            dsum=dealerAct(dealerCard)
        ret=getReturn(psum,dsum)

        for sanumber in sanumbers:
            rlist=returns[sanumber]
            rlist.append(ret)
            ave=sum(rlist)/len(rlist)
            s=sanumber // 2
            a=sanumber % 2
            Q[s,a]=ave
            if Q[s,1]>Q[s,0]:#Hitの価値が高い時
                piH[s]=1
            else:
                piH[s]=0
    
    difPi=piH-piBfr
    difQ=Q-QBfr
    
    table[i,0]=i
    table[i,1]=np.count_nonzero(difPi)
    table[i,2]=int(np.linalg.norm(difQ)*10000)    

In [139]:
#結果の評価


print(Q)
print(piH)

matrixA=np.zeros((11,11))
matrixN=np.zeros((11,11))
valueA=np.zeros((11,11))
valueN=np.zeros((11,11))

for i in range(10):
    matrixA[i+1,0]=i+12 #プレーヤー
    matrixN[i+1,0]=i+12 #プレーヤー
    valueA[i+1,0]=i+12 #プレーヤー
    valueN[i+1,0]=i+12 #プレーヤー
    matrixA[0,i+1]=i+2 #ディーラー
    matrixN[0,i+1]=i+2 #ディーラー
    valueA[0,i+1]=i+2 #プレーヤー
    valueN[0,i+1]=i+2 #プレーヤー

for i in range(200):
    sa=getStateAction(i*2)
    h=piH[i]
    if h>0.5:
        optAct='H'
        v=Q[i,1]
    else:
        optAct='S'
        v=Q[i,0]
    sa[3]=optAct
    sa.insert(0,i)
    print(sa)
        
    player=i%10
    dealer=(i//10)%10
    usableA=i//100
    if usableA==1:
        matrixA[player+1,dealer+1]=h
        valueA[player+1,dealer+1]=int(v*1000)
    else:
        matrixN[player+1,dealer+1]=h
        valueN[player+1,dealer+1]=int(v*1000)
    
print('With A')
print(matrixA)
print('Without A')
print(matrixN)

print('Value with A')
print(valueA)
print('Value without A')
print(valueN)



[[  0.00000000e+00   2.00000000e+02   2.23122000e+05]
 [  1.00000000e+00   3.20000000e+01   1.06006000e+05]
 [  2.00000000e+00   3.00000000e+01   6.44950000e+04]
 ..., 
 [  9.99700000e+03   0.00000000e+00   1.40000000e+01]
 [  9.99800000e+03   0.00000000e+00   1.50000000e+01]
 [  9.99900000e+03   0.00000000e+00   1.40000000e+01]]
[[-0.2965572  -0.24759652]
 [-0.28917301 -0.3025    ]
 [-0.28080627 -0.36122775]
 [-0.29014809 -0.40917725]
 [-0.28141955 -0.46979002]
 [-0.14787155 -0.5221    ]
 [ 0.11406322 -0.6168    ]
 [ 0.38749011 -0.719     ]
 [ 0.65143887 -0.85481452]
 [ 0.88029695 -1.        ]
 [-0.24518707 -0.22657508]
 [-0.23310382 -0.29270729]
 [-0.24894666 -0.35350509]
 [-0.24364643 -0.4027    ]
 [-0.25198537 -0.4621    ]
 [-0.10347888 -0.53989202]
 [ 0.15248634 -0.6110389 ]
 [ 0.40163858 -0.7312    ]
 [ 0.65126461 -0.8528    ]
 [ 0.88830634 -1.        ]
 [-0.22390171 -0.2207118 ]
 [-0.22178424 -0.26757324]
 [-0.20744513 -0.32680979]
 [-0.21125133 -0.39082642]
 [-0.20426193 -0.454